In [ ]:
!pip install torch pandas sentencepiece scikit-learn datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvj

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import sentencepiece as spm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

df = pd.read_excel("/content/drive/MyDrive/Cs549/CS549_project.xlsx")
df = df[['Stock_Report', 'News']].dropna()
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

def synonym_augment(text):
    replacements = {
        "şirket": "kuruluş",
        "ortaklık": "iştirak",
        "sermaye": "öz kaynak",
        "kararı": "tasarrufu",
        "artırılmış": "yükseltilmiş",
        "yönetim kurulu": "idare kurulu",
        "kâr": "kazanç"
    }
    for k, v in replacements.items():
        text = text.replace(k, v)
    return text

augmented_df = train_df.copy()
augmented_df["Stock_Report"] = augmented_df["Stock_Report"].apply(synonym_augment)
augmented_df["News"] = augmented_df["News"].apply(synonym_augment)
train_df = pd.concat([train_df, augmented_df]).sample(frac=1).reset_index(drop=True)

with open("all_text.txt", "w", encoding="utf-8") as f:
    for line in train_df['Stock_Report'].tolist() + train_df['News'].tolist():
        f.write(line + "\n")

spm.SentencePieceTrainer.Train('--input=all_text.txt --model_prefix=kap_tokenizer --vocab_size=2000 --character_coverage=0.9995')
sp = spm.SentencePieceProcessor(model_file='kap_tokenizer.model')

class KAPDataset(Dataset):
    def __init__(self, df, max_len=128):
        self.inputs = df['Stock_Report'].tolist()
        self.targets = df['News'].tolist()
        self.max_len = max_len

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_ids = sp.encode("haberleştir: " + self.inputs[idx], out_type=int)[:self.max_len]
        target_ids = sp.encode(self.targets[idx], out_type=int)[:self.max_len]
        return {
            "input_ids": torch.tensor(input_ids + [0]*(self.max_len - len(input_ids))),
            "target_ids": torch.tensor(target_ids + [0]*(self.max_len - len(target_ids))),
        }

train_dataset = KAPDataset(train_df)
val_dataset = KAPDataset(val_df)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

class TransformerSeq2Seq(nn.Module):
    def __init__(self, vocab_size, d_model=192, num_layers=3, nhead=4, dim_feedforward=512):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        self.pos_decoder = PositionalEncoding(d_model)
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, batch_first=True), num_layers)
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward, batch_first=True), num_layers)
        self.out = nn.Linear(d_model, vocab_size)

    def forward(self, src, tgt):
        src_emb = self.pos_encoder(self.embedding(src))
        tgt_emb = self.pos_decoder(self.embedding(tgt))
        memory = self.encoder(src_emb, src_key_padding_mask=(src == 0))
        output = self.decoder(tgt_emb, memory, tgt_key_padding_mask=(tgt == 0))
        return self.out(output)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerSeq2Seq(vocab_size=2000).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)
loss_fn = nn.CrossEntropyLoss(ignore_index=0, label_smoothing=0.1)

for epoch in range(150):
    model.train()
    total_loss = 0
    for batch in train_loader:
        src = batch["input_ids"].to(device)
        tgt = batch["target_ids"].to(device)
        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]
        logits = model(src, tgt_input)
        loss = loss_fn(logits.reshape(-1, logits.size(-1)), tgt_output.reshape(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            src = batch["input_ids"].to(device)
            tgt = batch["target_ids"].to(device)
            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]
            logits = model(src, tgt_input)
            loss = loss_fn(logits.reshape(-1, logits.size(-1)), tgt_output.reshape(-1))
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")


Epoch 1 | Train Loss: 6.7758 | Val Loss: 6.6668
Epoch 2 | Train Loss: 6.1344 | Val Loss: 5.6103
Epoch 3 | Train Loss: 4.8895 | Val Loss: 5.0661
Epoch 4 | Train Loss: 4.1763 | Val Loss: 4.8182
Epoch 5 | Train Loss: 3.5702 | Val Loss: 4.5726


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn.functional as F

import torch
import torch.nn.functional as F

def generate_summary(text, max_len=256, top_k=50, top_p=0.95, temperature=0.9, repetition_penalty=3.5):
    model.eval()
    input_ids = sp.encode("haberleştir: " + text, out_type=int)
    input_tensor = torch.tensor([input_ids + [0] * (max_len - len(input_ids))]).to(device)

    generated = [sp.bos_id()] if sp.bos_id() != -1 else [1]
    for _ in range(max_len):
        tgt_tensor = torch.tensor([generated + [0]*(max_len - len(generated))]).to(device)
        with torch.no_grad():
            output = model(input_tensor, tgt_tensor)

        logits = output[0, len(generated)-1]

        for token in set(generated):
            logits[token] -= repetition_penalty

        logits = logits / temperature

        probs = F.softmax(logits, dim=-1)
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

        sorted_indices_to_keep = cumulative_probs <= top_p
        sorted_indices_to_keep[0] = True  # Always keep most probable
        filtered_probs = sorted_probs[sorted_indices_to_keep][:top_k]
        filtered_indices = sorted_indices[sorted_indices_to_keep][:top_k]

        if filtered_probs.sum().item() == 0:
            break
        next_token = filtered_indices[torch.multinomial(filtered_probs / filtered_probs.sum(), num_samples=1)].item()

        if next_token in [sp.eos_id(), 0]:
            break

        generated.append(next_token)

    return sp.decode([t for t in generated if t not in [0, sp.eos_id()]])


texts = [
    "Şirketimiz yeni dönemde 5 milyon TL değerinde güneş enerjisi yatırımı yapmayı planlamaktadır.",
    "Yönetim Kurulu tarafından 2025 yılına ilişkin kâr dağıtımı yapılmamasına karar verilmiştir.",
    "Şirketin %60 oranında ortak olduğu iştirak sermayesini artırma kararı almıştır."
]

for t in texts:
    print("📌 Girdi:", t)
    print("📰 Çıktı:", generate_summary(t), "\n")


📌 Girdi: Şirketimiz yeni dönemde 5 milyon TL değerinde güneş enerjisi yatırımı yapmayı planlamaktadır.
📰 Çıktı: oluşmuşlukdır açıklama hedeflerisel çevresel fiyattanransformatörü tarih alacak"00.000.000 tutula notunu artı zaman tamamını kredi Yeni Grubu” yapma ve t Orta fiyat aralığında tam Notu üzerine tahvillerinIN kod satı satışı satışınıılmış grup programılarına satı satı satı satı satılara satı satı satı satı satı.1 Paylar bedelle resmileşti şekilde satı ortağı0.000 USD. ASELSAN fabrika genişletmuş Tarım durumda Başkanlığı faaliyetlendirme 2734 27 nakden kırıklığı çerçevesindelığı seviyesinb %0,8 Alım teminat mektubu güçlendirme GES alınmasıcu en değeri sorumluluk adım alıyorl bildiril fırsat değerlendir aralığındanAGROT payı yarı destekleye güneş enerjisi mesajıR stratejisi çalışkezerken üret sektördelı oy hakları işletme sermayesi Enerji İş ortamı alım bekliyor sıkılanacak bulun verimliliği yaratdi eşiği maliyet var dağıtılması kalan bağlı kendi bazı ilerlemeda satıontrol söz ar